In [1]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from factor_analyzer.factor_analyzer import calculate_kmo

In [2]:
plt.rcParams["font.family"] = 'AppleGothic' if sys.platform == 'darwin' else 'Malgun Gothic'
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.unicode_minus"] = False

## 데이터 불러오기

In [4]:
origin=read_excel('행정동병합테스트22(개발제한행정동제외).xlsx')
origin2=read_excel('여가생활지수1.xlsx')
origin3=read_excel('사회적약자_지원_인프라_밀집도1.xlsx')
origin4=read_excel('접근성지수1.xlsx')
origin5=read_excel('환경자족성지수1.xlsx')
origin.head(1)

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,2154,437,337.55,4,0.352898,0,32,0,3,4552.529183


In [5]:
origin.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,2154,437,337.55,4,0.352898,0,32,0,3,4552.529183
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,1808,267,60.79,0,0.063554,0,27,0,12,7611.382114
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,626,110,956.51,3,1.000000,0,16,0,5,1728.859060
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,1814,311,0.00,3,0.000000,10,34,0,5,4130.837004
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,3495,491,1.14,3,0.001192,10,54,0,5,1990.755355


## 병합할 지수를 `읍면동` 주소를 기준으로 병합

### 병합에 사용될 컬럼 추출 1

In [6]:
origin2.columns[-1]

'여가생활지수minmax'

In [7]:
origin2_2=origin2.filter(['읍면동명','여가생활지수minmax'])
origin2_2

,읍면동명,여가생활지수minmax
0,청운효자동,0.002648
1,사직동,0.006464
2,삼청동,0.005332
3,부암동,0.001699
4,평창동,0.000529
...,...,...
401,성내제2동,0.045969
402,성내제3동,0.041755
403,길동,0.029283
404,둔촌제1동,0.000000


### 병합에 사용될 컬럼 추출 2

In [8]:
origin3.columns[-1]

'사회적약자_지원_인프라_밀집도minmax'

In [10]:
origin3_3=origin3.filter(['읍면동명','사회적약자_지원_인프라_밀집도minmax'])
origin3_3

,읍면동명,사회적약자_지원_인프라_밀집도minmax
0,청운효자동,0.055189
1,사직동,0.033809
2,삼청동,0.072924
3,부암동,0.054786
4,평창동,0.059060
...,...,...
401,성내제2동,0.049013
402,성내제3동,0.060913
403,길동,0.046137
404,둔촌제1동,0.806447


### 병합에 사용될 컬럼 추출 3

In [11]:
origin4.columns[-1]

'접근성지수minmax'

In [12]:
origin4_4=origin4.filter(['읍면동명','접근성지수minmax'])
origin4_4

,읍면동명,접근성지수minmax
0,청운효자동,0.114798
1,사직동,0.333275
2,삼청동,0.342620
3,부암동,0.157400
4,평창동,0.135935
...,...,...
401,성내제2동,0.108987
402,성내제3동,0.079180
403,길동,0.064183
404,둔촌제1동,0.000000


### 병합에 사용될 컬럼 추출 4

In [13]:
origin5.columns[-1]

'환경자족성지수minmax'

In [14]:
origin5_5=origin5.filter(['읍면동명','환경자족성지수minmax'])
origin5_5

,읍면동명,환경자족성지수minmax
0,청운효자동,0.212495
1,사직동,0.038269
2,삼청동,0.602143
3,부암동,0.602143
4,평창동,0.602861
...,...,...
401,성내제2동,0.240970
402,성내제3동,0.542149
403,길동,0.002587
404,둔촌제1동,0.421500


### `읍면동` 데이터를 기준으로 지수 병합 1

In [15]:
tmp=merge(origin,origin2_2,how='left')
tmp.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도,여가생활지수minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,437,337.55,4,0.352898,0,32,0,3,4552.529183,0.002648
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,267,60.79,0,0.063554,0,27,0,12,7611.382114,0.006464
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,110,956.51,3,1.000000,0,16,0,5,1728.859060,0.005332
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,311,0.00,3,0.000000,10,34,0,5,4130.837004,0.001699
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,491,1.14,3,0.001192,10,54,0,5,1990.755355,0.000529


### `읍면동` 데이터를 기준으로 지수 병합 2

In [16]:
tmp1=merge(tmp,origin3_3,how='left')
tmp1.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도,여가생활지수minmax,사회적약자_지원_인프라_밀집도minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,337.55,4,0.352898,0,32,0,3,4552.529183,0.002648,0.055189
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,60.79,0,0.063554,0,27,0,12,7611.382114,0.006464,0.033809
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,956.51,3,1.000000,0,16,0,5,1728.859060,0.005332,0.072924
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,0.00,3,0.000000,10,34,0,5,4130.837004,0.001699,0.054786
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,1.14,3,0.001192,10,54,0,5,1990.755355,0.000529,0.059060


### `읍면동` 데이터를 기준으로 지수 병합 3

In [17]:
tmp2=merge(tmp1,origin4_4,how='left')
tmp2.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도,여가생활지수minmax,사회적약자_지원_인프라_밀집도minmax,접근성지수minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,4,0.352898,0,32,0,3,4552.529183,0.002648,0.055189,0.114798
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0,0.063554,0,27,0,12,7611.382114,0.006464,0.033809,0.333275
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,3,1.000000,0,16,0,5,1728.859060,0.005332,0.072924,0.342620
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,3,0.000000,10,34,0,5,4130.837004,0.001699,0.054786,0.157400
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,3,0.001192,10,54,0,5,1990.755355,0.000529,0.059060,0.135935


### `읍면동` 데이터를 기준으로 지수 병합 4

In [18]:
tmp3=merge(tmp2,origin5_5,how='left')
tmp3.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도,여가생활지수minmax,사회적약자_지원_인프라_밀집도minmax,접근성지수minmax,환경자족성지수minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0.352898,0,32,0,3,4552.529183,0.002648,0.055189,0.114798,0.212495
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0.063554,0,27,0,12,7611.382114,0.006464,0.033809,0.333275,0.038269
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,1.000000,0,16,0,5,1728.859060,0.005332,0.072924,0.342620,0.602143
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,0.000000,10,34,0,5,4130.837004,0.001699,0.054786,0.157400,0.602143
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,0.001192,10,54,0,5,1990.755355,0.000529,0.059060,0.135935,0.602861


## 최종 데이터 확인 및 xlsx 파일 생성

In [19]:
tmp3.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '인구밀도', '여가생활지수minmax', '사회적약자_지원_인프라_밀집도minmax',
       '접근성지수minmax', '환경자족성지수minmax'],
      dtype='object')

In [20]:
tmp3.to_excel('행정동병합최종_반포본동.xlsx',index=False)

In [21]:
tmp3.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도,여가생활지수minmax,사회적약자_지원_인프라_밀집도minmax,접근성지수minmax,환경자족성지수minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0.352898,0,32,0,3,4552.529183,0.002648,0.055189,0.114798,0.212495
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0.063554,0,27,0,12,7611.382114,0.006464,0.033809,0.333275,0.038269
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,1.000000,0,16,0,5,1728.859060,0.005332,0.072924,0.342620,0.602143
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,0.000000,10,34,0,5,4130.837004,0.001699,0.054786,0.157400,0.602143
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,0.001192,10,54,0,5,1990.755355,0.000529,0.059060,0.135935,0.602861
